In [58]:
import requests
import string
from bs4 import BeautifulSoup
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from unicodedata import numeric
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [59]:
URL = "https://www.allrecipes.com/recipe/11679/homemade-mac-and-cheese/"

In [60]:
class Ingredient:

  def __init__(self,fullname=None,main_ingredients=None,quantity_type=None,quantity = None,units=None,descriptor=None,preparation=None,ingredient_type=None):  
    self.fullname = fullname
    self.main_ingredients = main_ingredients
    self.quantity_type = quantity_type
    self.quantity = quantity
    self.units = units
    self.descriptor = descriptor
    self.preparation = preparation
    self.ingredient_type = ingredient_type

  def print_ingredient(self):
    print('Full Name: ',self.fullname)
    print("Main Ingredient: ",self.main_ingredients)
    print("Quantity Type: ",self.quantity_type)
    print("Quantity: ",self.quantity)
    print("Units: ",self.units)
    print('Descriptor: ',self.descriptor)
    print('Preparation: ',self.preparation,'\n')

In [61]:
class Step:
  def __init__(self,full_step=None,ingredients=None,tools=None,methods = None,times=None):  
    self.full_step = full_step
    self.ingredients = ingredients
    self.tools = tools
    self.methods = methods
    self.times = times
  
  def print_steps(self):
    print('Full Step: ',self.full_step)
    print("Ingredients: ",self.ingredients)
    print("Tools: ",self.tools)
    print("Methods: ",self.methods)
    print("Times: ",self.times,'\n')

In [62]:
def get_ingredients_tools_methods_steps(URL):
  page = requests.get(URL)
  soup = BeautifulSoup(page.content, "html.parser")
  ingredient_parse = soup.find_all("input", class_="checkbox-list-input")
  direction_parse = soup.find_all("div", class_="paragraph")
  unicode_quantities = []
  unicode_units = []
  unicode_ingredients = []
  unicode_type = []

  #----------------------------Parsing Ingredients--------------------------#
  for ingredient in ingredient_parse:
    val = ingredient.attrs
    if u'data-quantity' in val.keys():
        unicode_quantities.append(val[u'data-init-quantity'])
    if u'data-unit' in val.keys():
        unicode_units.append(val[u'data-unit'])
    if u'data-ingredient' in val.keys():
        unicode_ingredients.append(val[u'data-ingredient'])
    if u'data-ingredient' in val.keys():
        unicode_type.append(val[u'data-unit_family'])

  quantities = unicode_quantities
  units = unicode_units
  ingredients = unicode_ingredients
  quantity_type = unicode_type
  descriptor  = [None] * len(ingredients)
  preparation = [None] * len(ingredients)
  main_ingredients = [0] * len(ingredients)

  def preprocess(sent):
    sent = nltk.word_tokenize(sent)
    sent = nltk.pos_tag(sent)
    return sent
  
  for i in range(len(ingredients)):
    tokens = preprocess(ingredients[i])
    for token in tokens:
      if token[1] == 'JJ':
        descriptor[i] = token[0]
      elif token[1] in ['VBD','VBN']:
        preparation[i] = token[0]
      elif token[1] in ['NN','NNS','NNP','NNPS']:
        main_ingredients[i] = token[0]

  Ingredients = [0] * len(ingredients)
  for i in range(len(ingredients)):
    new_ingredient = Ingredient(fullname=ingredients[i],main_ingredients=main_ingredients[i],quantity_type=quantity_type[i],quantity = quantities[i],units=units[i],descriptor=descriptor[i],preparation=preparation[i],ingredient_type=None)
    Ingredients[i] = new_ingredient
  
  #----------------------------Ingredients are ready--------------------------#


  #----------------------------Half Parsing Directions--------------------------#
  directions = []

  for step in direction_parse:
    step_text = step.text
    step_text.encode("ascii", "ignore").strip()
    directions.append(step_text)

  #--------------------------Half Parsed Directions are ready------------#

  #----------------------------Parsing Tools--------------------------#
  with open("/content/drive/MyDrive/NLP2/list_of_tools.txt") as file:
      lines = file.readlines()
      list_of_tools = [line.rstrip() for line in lines]

  Tools = []
  for i in range(len(directions)):
    direction_lower = directions[i]
    for tool in list_of_tools:
      if tool in direction_lower:
        Tools.append(tool)

  #----------------------------Tools are ready--------------------------#

  #----------------------------Parsing Methods--------------------------#
  Cooking_Methods = []
  with open("/content/drive/MyDrive/NLP2/list_of_cooking_methods.txt") as file:
      lines = file.readlines()
      list_of_cooking_methods = [line.rstrip() for line in lines]
  
  lemmatizer = WordNetLemmatizer()
  for i in range(len(directions)):
    direction_lower = directions[i].lower()
    for cooking_method in list_of_cooking_methods:
      if cooking_method in direction_lower or lemmatizer.lemmatize(cooking_method) in direction_lower:
        Cooking_Methods.append(cooking_method)
        
  
  #----------------------------Methods are ready--------------------------#

  #----------------------------Parsing Directions--------------------------#
  Steps = []

  for i in range(len(directions)):
    direction_lower = directions[i].lower()
    step_wise_ingredients = []
    step_wise_tools = []
    step_wise_methods = []
    step_wise_times = ''
    
    for tool in Tools:
      if tool in direction_lower and tool not in step_wise_tools:
        step_wise_tools.append(tool)
    
    for method in Cooking_Methods:
      if method in direction_lower and method not in step_wise_methods:
        step_wise_methods.append(method)
    
    for ingredient in Ingredients:
      if ingredient.main_ingredients in direction_lower and ingredient.fullname not in step_wise_ingredients :
        step_wise_ingredients.append(ingredient.fullname)
    
    time_keywords = 'minutes|seconds|hours|secs|hrs|mins|min'
    if (re.search(time_keywords,direction_lower)):
      index = re.search(time_keywords,direction_lower).start()
      new_str = direction_lower[:index]
      step_wise_times+= str(new_str.split()[-1])
      unit = re.findall(time_keywords,direction_lower)[-1]
      step_wise_times += ' '+str(unit)
    new_step = Step (full_step=directions[i],ingredients=step_wise_ingredients,tools=step_wise_tools,methods = step_wise_methods,times=step_wise_times)
    Steps.append(new_step)
    
  return Ingredients, Tools, Cooking_Methods, Steps
 #----------------------------Directions are ready--------------------------#



In [63]:
Ingredients, Tools, Cooking_Methods, Steps = get_ingredients_tools_methods_steps(URL)

In [64]:
for ingredient in Ingredients:
  ingredient.print_ingredient()

Full Name:  uncooked elbow macaroni
Main Ingredient:  macaroni
Quantity Type:  weight
Quantity:  8
Units:  ounces
Descriptor:  uncooked
Preparation:  None 

Full Name:  shredded sharp Cheddar cheese
Main Ingredient:  cheese
Quantity Type:  volumetric
Quantity:  2
Units:  cups
Descriptor:  sharp
Preparation:  shredded 

Full Name:  grated Parmesan cheese
Main Ingredient:  cheese
Quantity Type:  volumetric
Quantity:  0.5
Units:  cup
Descriptor:  None
Preparation:  grated 

Full Name:  milk
Main Ingredient:  milk
Quantity Type:  volumetric
Quantity:  3
Units:  cups
Descriptor:  None
Preparation:  None 

Full Name:  butter
Main Ingredient:  butter
Quantity Type:  volumetric
Quantity:  0.25
Units:  cup
Descriptor:  None
Preparation:  None 

Full Name:  all-purpose flour
Main Ingredient:  flour
Quantity Type:  volumetric
Quantity:  2.5
Units:  tablespoons
Descriptor:  all-purpose
Preparation:  None 

Full Name:  butter
Main Ingredient:  butter
Quantity Type:  volumetric
Quantity:  2
Units:  

In [65]:
for tool in Tools:
  print(tool)

saucepan
pan
casserole
skillet


In [66]:
for method in Cooking_Methods:
  print(method)

roux
brown
bake


In [67]:
for step in Steps:
  step.print_steps()

Full Step:  Cook macaroni according to the package directions. Drain. 
Ingredients:  ['uncooked elbow macaroni']
Tools:  []
Methods:  []
Times:   

Full Step:  In a saucepan, melt butter or margarine over medium heat. Stir in enough flour to make a roux. Add milk to roux slowly, stirring constantly. Stir in cheeses, and cook over low heat until cheese is melted and the sauce is a little thick. Put macaroni in large casserole dish, and pour sauce over macaroni. Stir well. 
Ingredients:  ['uncooked elbow macaroni', 'shredded sharp Cheddar cheese', 'grated Parmesan cheese', 'milk', 'butter', 'all-purpose flour']
Tools:  ['saucepan', 'pan', 'casserole']
Methods:  ['roux']
Times:   

Full Step:  Melt butter or margarine in a skillet over medium heat. Add breadcrumbs and brown. Spread over the macaroni and cheese to cover. Sprinkle with a little paprika. 
Ingredients:  ['uncooked elbow macaroni', 'shredded sharp Cheddar cheese', 'grated Parmesan cheese', 'butter', 'bread crumbs', 'paprika']


In [68]:
lemmatizer = WordNetLemmatizer()
lemmatizer.lemmatize('baking')

'baking'